In [14]:
import pandas as pd
from tensorflow import keras
from gensim.models import Phrases
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import utils
import pickle

In [15]:
# function to clean original tweets
def clean_text(df):
    for i in range(df['text'].shape[0]):
        df['text'][i] = " ".join([word for word in df['text'][i].split()
                                    if 'http' not in word
                                    and not word.startswith('@')
                                    # and not word.startswith('#')
                                    and word != 'RT'
                                    ])
    df['text'] = df['text'].str.replace('[^A-Za-z ]+', '')

    return df

In [16]:
# process data
data = pd.read_csv('/Users/BabeChris/Desktop/4th/STA414/A1/DataFest2020/Training Data/Sentiment.csv')
data = data[['text','sentiment']]
data = data.replace('Neutral', 0)
data = data.replace('Positive', 1)
data = data.replace('Negative', -1)
data = clean_text(data)

# data we use
tweet, label = data['text'], data['sentiment']

/Users/BabeChris/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
# Convert a collection of raw documents to a matrix of TF-IDF features
vectorizer = TfidfVectorizer(ngram_range=(1,2))
corpus = [line for line in tweet]

# Learn the vocabulary dictionary and return document-term matrix.
tfidf = vectorizer.fit(corpus)
X = vectorizer.fit_transform(corpus)

In [52]:
# R-F model & fitting
clf = RandomForestClassifier()
clf.fit(X[:-100], label.iloc[:-100])

print(clf.score(X[:-1000], label.iloc[:-1000]))
print(clf.score(X[-1000:], label.iloc[-1000:]))

/Users/BabeChris/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9212959366016626
0.813


In [53]:
# Saving model
model_path = '/Users/BabeChris/Desktop/UofT/4th/DataFest June/Models/RFmodel.pickle'
vectorizer_path = '/Users/BabeChris/Desktop/UofT/4th/DataFest June/Models/RFvectorizer.pickle'
pickle.dump(clf, open(model_path, 'wb'))
pickle.dump(tfidf, open(vectorizer_path, 'wb'))

In [56]:
# Load the model
model_path = '/Users/BabeChris/Desktop/UofT/4th/DataFest June/Models/RFmodel.pickle'
vectorizer_path = '/Users/BabeChris/Desktop/UofT/4th/DataFest June/Models/RFvectorizer.pickle'

model = pickle.load(open(model_path,'rb'))
vectorizer = pickle.load(open(vectorizer_path,'rb'))

pred = model.predict(vectorizer.transform(["i have got a new phone. its from Apple.. and i love it!"]))[0]
print ("predicted class:", pred)

predicted class: 1
